Lets start by dividing the task into some components that are necessary to implement:

1. Data Preprocessing 
2. Text-Based Product Recommendation
3. Image-Based Product Recommendation
4. Integration - Combine recommendation systems and get a list of final recommendations
5. Evaluation - Fine tunning the parameters

In [ ]:
# Data Preprocessing